In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('SparkSQL-test').getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/09 12:57:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
stocks = [
    ('Google', 'GOOGL', 'USA', 2984, 'USD'), 
    ('Netflix', 'NFLX', 'USA', 645, 'USD'),
    ('Amazon', 'AMZN', 'USA', 3518, 'USD'),
    ('Tesla', 'TSLA', 'USA', 1222, 'USD'),
    ('Tencent', '0700', 'Hong Kong', 483, 'HKD'),
    ('Toyota', '7203', 'Japan', 2006, 'JPY'),
    ('Samsung', '005930', 'Korea', 70600, 'KRW'),
    ('Kakao', '035720', 'Korea', 125000, 'KRW'),
]

In [3]:
stockSchema = ['name', 'ticker', 'country', 'price', 'currency']

df = spark.createDataFrame(data = stocks,
                           schema = stockSchema)

In [4]:
df.dtypes

[('name', 'string'),
 ('ticker', 'string'),
 ('country', 'string'),
 ('price', 'bigint'),
 ('currency', 'string')]

In [5]:
df.show()

+-------+------+---------+------+--------+
|   name|ticker|  country| price|currency|
+-------+------+---------+------+--------+
| Google| GOOGL|      USA|  2984|     USD|
|Netflix|  NFLX|      USA|   645|     USD|
| Amazon|  AMZN|      USA|  3518|     USD|
|  Tesla|  TSLA|      USA|  1222|     USD|
|Tencent|  0700|Hong Kong|   483|     HKD|
| Toyota|  7203|    Japan|  2006|     JPY|
|Samsung|005930|    Korea| 70600|     KRW|
|  Kakao|035720|    Korea|125000|     KRW|
+-------+------+---------+------+--------+



In [6]:
df.collect()

[Row(name='Google', ticker='GOOGL', country='USA', price=2984, currency='USD'),
 Row(name='Netflix', ticker='NFLX', country='USA', price=645, currency='USD'),
 Row(name='Amazon', ticker='AMZN', country='USA', price=3518, currency='USD'),
 Row(name='Tesla', ticker='TSLA', country='USA', price=1222, currency='USD'),
 Row(name='Tencent', ticker='0700', country='Hong Kong', price=483, currency='HKD'),
 Row(name='Toyota', ticker='7203', country='Japan', price=2006, currency='JPY'),
 Row(name='Samsung', ticker='005930', country='Korea', price=70600, currency='KRW'),
 Row(name='Kakao', ticker='035720', country='Korea', price=125000, currency='KRW')]

In [7]:
df.createOrReplaceTempView('stock') # Temporary View 생성


In [8]:
spark.sql('''SELECT name
          FROM stock''').show()

+-------+
|   name|
+-------+
| Google|
|Netflix|
| Amazon|
|  Tesla|
|Tencent|
| Toyota|
|Samsung|
|  Kakao|
+-------+



In [9]:
spark.sql('''SELECT name, price
          FROM stock''').show()

+-------+------+
|   name| price|
+-------+------+
| Google|  2984|
|Netflix|   645|
| Amazon|  3518|
|  Tesla|  1222|
|Tencent|   483|
| Toyota|  2006|
|Samsung| 70600|
|  Kakao|125000|
+-------+------+



In [10]:
spark.sql(
    '''
    select name, ticker, price
    from stock
    where country=="Korea"
    '''
).show()

+-------+------+------+
|   name|ticker| price|
+-------+------+------+
|Samsung|005930| 70600|
|  Kakao|035720|125000|
+-------+------+------+



In [11]:
spark.sql(
    '''
    SELECT name, ticker, price
    from stock
    where price > 2000
    and country == "USA"
    '''
).show()

+------+------+-----+
|  name|ticker|price|
+------+------+-----+
|Google| GOOGL| 2984|
|Amazon|  AMZN| 3518|
+------+------+-----+



In [12]:
spark.sql(
    '''
    SELECT name, ticker, name
    from stock
    where country like 'U__'
    '''
).show()

+-------+------+-------+
|   name|ticker|   name|
+-------+------+-------+
| Google| GOOGL| Google|
|Netflix|  NFLX|Netflix|
| Amazon|  AMZN| Amazon|
|  Tesla|  TSLA|  Tesla|
+-------+------+-------+



In [13]:
spark.sql(
    '''
    SELECT name, ticker, name
    from stock
    where country like 'U__' and name not like "_e%"
    '''
).show()

+------+------+------+
|  name|ticker|  name|
+------+------+------+
|Google| GOOGL|Google|
|Amazon|  AMZN|Amazon|
+------+------+------+



In [14]:
# +-------+------+---------+------+--------+
# |   name|ticker|  country| price|currency|
# +-------+------+---------+------+--------+
# | Google| GOOGL|      USA|  2984|     USD|
# |Netflix|  NFLX|      USA|   645|     USD|
# | Amazon|  AMZN|      USA|  3518|     USD|
# |  Tesla|  TSLA|      USA|  1222|     USD|
# |Tencent|  0700|Hong Kong|   483|     HKD|
# | Toyota|  7203|    Japan|  2006|     JPY|
# |Samsung|005930|    Korea| 70600|     KRW|
# |  Kakao|035720|    Korea|125000|     KRW|
# +-------+------+---------+------+--------+

spark.sql(
    """
    select * 
    from stock
    where price between 1000 and 10000
    """
).show()

+------+------+-------+-----+--------+
|  name|ticker|country|price|currency|
+------+------+-------+-----+--------+
|Google| GOOGL|    USA| 2984|     USD|
|Amazon|  AMZN|    USA| 3518|     USD|
| Tesla|  TSLA|    USA| 1222|     USD|
|Toyota|  7203|  Japan| 2006|     JPY|
+------+------+-------+-----+--------+



In [22]:
from pyspark.sql.types import StringType, FloatType, StructType, StructField

# 주당 순이익 
earnings = [
    ('Google', 27.99, 'USD'), 
    ('Netflix', 2.56, 'USD'),
    ('Amazon', 6.12, 'USD'),
    ('Tesla', 1.86, 'USD'),
    ('Tencent', 11.01, 'HKD'),
    ('Toyota', 224.82, 'JPY'),
    ('Samsung', 1780., 'KRW'),
    ('Kakao', 705., 'KRW')
]

earnings_schema = StructType([
    StructField('name', StringType(), True),
    StructField('eps', FloatType(), True),
    StructField('currency', StringType(), True)
])

In [23]:
df2 = spark.createDataFrame(data=earnings, schema=earnings_schema)
df2.createOrReplaceTempView('earnings')

In [27]:
df2.dtypes

[('name', 'string'), ('eps', 'float'), ('currency', 'string')]

In [26]:
spark.sql(
    """
    select *
    from earnings
    """
).show()

+-------+------+--------+
|   name|   eps|currency|
+-------+------+--------+
| Google| 27.99|     USD|
|Netflix|  2.56|     USD|
| Amazon|  6.12|     USD|
|  Tesla|  1.86|     USD|
|Tencent| 11.01|     HKD|
| Toyota|224.82|     JPY|
|Samsung|1780.0|     KRW|
|  Kakao| 705.0|     KRW|
+-------+------+--------+



In [28]:
spark.sql(
    """
    show tables
    """
).show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         | earnings|       true|
|         |    stock|       true|
+---------+---------+-----------+



In [33]:
spark.sql(
    """select *
    from stock 
    join earnings on stock.name  = earnings.name"""
).show()

+-------+------+---------+------+--------+-------+------+--------+
|   name|ticker|  country| price|currency|   name|   eps|currency|
+-------+------+---------+------+--------+-------+------+--------+
| Amazon|  AMZN|      USA|  3518|     USD| Amazon|  6.12|     USD|
| Google| GOOGL|      USA|  2984|     USD| Google| 27.99|     USD|
|  Kakao|035720|    Korea|125000|     KRW|  Kakao| 705.0|     KRW|
|Netflix|  NFLX|      USA|   645|     USD|Netflix|  2.56|     USD|
|Samsung|005930|    Korea| 70600|     KRW|Samsung|1780.0|     KRW|
|Tencent|  0700|Hong Kong|   483|     HKD|Tencent| 11.01|     HKD|
|  Tesla|  TSLA|      USA|  1222|     USD|  Tesla|  1.86|     USD|
| Toyota|  7203|    Japan|  2006|     JPY| Toyota|224.82|     JPY|
+-------+------+---------+------+--------+-------+------+--------+



In [35]:
spark.sql(
    """select stock.name, round(stock.price/earnings.eps) as PER
    from stock 
    join earnings on stock.name  = earnings.name"""
).show()

+-------+-----+
|   name|  PER|
+-------+-----+
| Amazon|575.0|
| Google|107.0|
|  Kakao|177.0|
|Netflix|252.0|
|Samsung| 40.0|
|Tencent| 44.0|
|  Tesla|657.0|
| Toyota|  9.0|
+-------+-----+

